# Router Chain
Select the next chain based on a parameter. It's possible to follow different paths.

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

phys_template = ChatPromptTemplate.from_template("""You are a very intelligent physics teacher.
You are great at answering physics questions in a concise and easy-to-understand way.
When you don't know the answer to a question, you admit that you don't know.

Here is a question: {input}""")

math_template = ChatPromptTemplate.from_template("""You are a very good mathematician.
You are great at answering math questions.
You are so good because you can break down difficult problems into their component parts,
answer the component parts, and then put them together
to answer the larger question.

Here is a question: {input}""")

hist_template = ChatPromptTemplate.from_template("""You are a very good historian.
You have an excellent knowledge and understanding of people,
events and contexts from a variety of historical periods.
You have the ability to think, reflect, debate, discuss and
evaluate the past. You have a respect for historical evidence
and the ability to use it to support your explanations and judgements.

Here is a question: {input}""")

prompts_data = [
    {
        'name': 'physics',
        'description': 'ideal for answers questions about physics',
        'prompt_template': phys_template,
    },
    {
        'name': 'mathematics',
        'description': 'ideal for answers questions about mathematics',
        'prompt_template': math_template,
    },
    {
        'name': 'history',
        'description': 'ideal for answers questions about history',
        'prompt_template': hist_template
    }
]

llm = ChatOpenAI(model="gpt-3.5-turbo")

destination_chains = {}

for info in prompts_data:
    chain = LLMChain(llm=llm, prompt=info['prompt_template'], verbose=True)
    destination_chains[info['name']] = chain
    
destinations = [f'{p["name"]}: {p["description"]}' for p in prompts_data]

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)

prompt_template = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

# If no matching chains found, it falls back to the default chain
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=llm, prompt=default_prompt, verbose=True)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

chain.invoke({'input': 'What is the speed of light?'})

/Users/viniciusfinger/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
physics: {'input': 'What is the speed of light?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a very intelligent physics teacher.
You are great at answering physics questions in a concise and easy-to-understand way.
When you don't know the answer to a question, you admit that you don't know.

Here is a question: What is the speed of light?

> Finished chain.

> Finished chain.


{'input': 'What is the speed of light?',
 'text': 'The speed of light in a vacuum is approximately 299,792,458 meters per second, or about 186,282 miles per second.'}

In [6]:
chain.invoke({'input': 'What is the second industrial revolution?'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
history: {'input': 'What is the second industrial revolution?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a very good historian.
You have an excellent knowledge and understanding of people,
events and contexts from a variety of historical periods.
You have the ability to think, reflect, debate, discuss and
evaluate the past. You have a respect for historical evidence
and the ability to use it to support your explanations and judgements.

Here is a question: What is the second industrial revolution?

> Finished chain.

> Finished chain.


{'input': 'What is the second industrial revolution?',
 'text': 'The Second Industrial Revolution, also known as the Technological Revolution, was a period of rapid industrial growth and innovation that took place in the late 19th and early 20th centuries. It was characterized by advancements in technology, such as the widespread use of electricity, the development of the telephone and telegraph, the invention of the light bulb, and the expansion of the railroad network.\n\nKey industries that flourished during this time included steel production, oil refining, and the manufacturing of automobiles and consumer goods. The Second Industrial Revolution also saw the rise of large corporations and the growth of urban centers as people moved from rural areas to cities in search of work.\n\nOverall, the Second Industrial Revolution transformed economies and societies around the world, leading to increased productivity, higher living standards, and significant changes in the way people lived a